In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password

In [2]:
zika_path = "Resources/ZikaData2015.xlsx"
df_zika_init = pd.read_excel(zika_path)
demograph_path = "Resources/PEP_2018_PEPTCOMP_with_ann.csv"
df_demo_raw_init = pd.read_csv(demograph_path)
df_demo_raw = df_demo_raw_init.iloc[1:]
df_zika_raw = df_zika_init.iloc[1:]

In [3]:
df_demo_raw.rename (columns={'GEO.display-label': 'States'}, inplace=True)
df_demo_select = df_demo_raw[["States","births4201072018","deaths4201072018"]]

C:\Users\Admin\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
df_merged = df_demo_select.merge(df_zika_raw, on="States")

In [5]:
df_merged_drop = df_merged.drop(columns = ["Unnamed: 2", "Unnamed: 4", "Presumptive viremic blood donors†(N=0)"])

In [6]:
df_merged_renamed = df_merged_drop.rename(columns = {"States":"State","births4201072018":"Births","deaths4201072018":"Deaths","Symptomatic disease cases*(N=62)":"Zika_Cases"})
df_merged_renamed.head()

,State,Births,Deaths,Zika_Cases
0,Alabama,484329,418734,0
1,Arizona,703872,434572,0
2,Arkansas,314960,251951,2
3,California,4078895,2094459,13
4,Colorado,543471,289139,0


In [7]:
#Export to SQL
#Make sure to run queries.sql to set up database
#Remember to create config.py with user and password
rds_connection_string = user + ":" + password + "@127.0.0.1/state_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [8]:
engine.table_names()

['state_data']

In [10]:
df_merged_renamed.to_sql(name='state_data', con=engine, if_exists='append', index=False)

In [11]:
pd.read_sql_query('select * from state_data', con=engine).head()

,State,Births,Deaths,Zika_Cases
0,Alabama,484329,418734,0
1,Arizona,703872,434572,0
2,Arkansas,314960,251951,2
3,California,4078895,2094459,13
4,Colorado,543471,289139,0
